In [1]:
import os
from pathlib import Path
import json
import pandas as pd
from ipyleaflet import Map, Choropleth
from branca.colormap import linear

### Load data
appDir = Path(os.path.abspath(''))
# used https://mapshaper.org/ to simplify the GeoJSON file, reducing its size using the Visvalingam / weighted area method with a 1% zoom level
with open(appDir / 'OutcodeCoordinates_compressed.json', 'r') as f:
    outcodeCoordinates = json.load(f)
summary = pd.read_csv(appDir / 'summary.csv')
yearBins = list(summary['YearBin'].unique())

input_yearBin = tuple()
input_switch = False
input_statistic = 'mean'

# function to filter the summary dataset and return a lookup dictionary with a key for each Outcode
# select all time periods if none are selected
if input_yearBin == tuple():
    filter = yearBins
else:
    filter = list(input_yearBin)

# logic for comparing summary stastics between time periods
if input_switch:
    minYearBin = filter[0]
    maxYearBin = filter[-1]
    dfMin = summary[summary['YearBin'] == minYearBin][['Outcode', input_statistic]]
    dfMax = summary[summary['YearBin'] == maxYearBin][['Outcode', input_statistic]]
    df = pd.merge(dfMin, dfMax, how="inner", on="Outcode")
    df['diff'] = df[input_statistic + '_y'] - df[input_statistic + '_x']
    df = df.set_index('Outcode')
    df['decile'] = pd.qcut(df['diff'], 10, labels=False)
    choroData = df['decile'].to_dict()
# if not comparing time periods then just show summary statistic
else:
    df = summary[summary['YearBin'].isin(filter)].set_index('Outcode')
    df['decile'] = pd.qcut(df[input_statistic], 10, labels=False)
    choroData = df['decile'].to_dict()

# create a Map object and add a Choropleth layer to it
m = Map(center=(54.00366, -2.547855), zoom=5.5)

layer = Choropleth(
            geo_data=outcodeCoordinates,
            choro_data=choroData,
            key_on='id',
            colormap=linear.viridis,
            border_color='black',
            style={'fillOpacity': 0.8, 'dashArray': '5, 5'}
        )
        
m.add(layer)

print("Hello")

# m

Map(center=[54.00366, -2.547855], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …